## Libraries Import:

In [1]:
import requests
import json
from datetime import datetime, date, timedelta
import pandas as pd
import numpy as np
import pyinputplus as pyip    # trying out this! -- pip install pyinputplus

## Getting User Input:

### 1. number of stocks and list of stocks

In [2]:
# list of stocks:
input_list = ['PYPL','TSLA','U','AMD','NFLX','SQ','NIO','ILMN','AMZN','GOOG']
user_option = []


# get user input - to get the number of stocks, from 1 to 10
while True:
    num = input('Enter the number of stocks (from 1 to 10): ')
    try:
        num = int(num)
    except ValueError:
        print('WARNING: please enter value ranges from 1 to 10 only!')
        continue
    if (num<1) or (num>10):
        print('WARNING: please enter value ranges from 1 to 10 only!')
        continue
    break

    
# get user input - to get the list of stocks
if num == 10:
    user_option = input_list
else:
    for i in range(1, num+1):
        print('*'*45)
        print(f'Please choose your stock no. {i} from the following stock list:')
        for stock in input_list:
            print(stock)
        while True:
            option = input('Enter your stock: ')
            try:
                input_list.remove(option)
                user_option.append(option)
                print('*'*45)
                print()
            except ValueError:
                print('WARNING: please only enter stock listed from the list above!')
                continue
            break
            
print(f'This is your list of stocks: {user_option}')

Enter the number of stocks (from 1 to 10): 15
Enter the number of stocks (from 1 to 10): 5
*********************************************
Please choose your stock no. 1 from the following stock list:
PYPL
TSLA
U
AMD
NFLX
SQ
NIO
ILMN
AMZN
GOOG
Enter your stock: SQ
*********************************************

*********************************************
Please choose your stock no. 2 from the following stock list:
PYPL
TSLA
U
AMD
NFLX
NIO
ILMN
AMZN
GOOG
Enter your stock: AMZN
*********************************************

*********************************************
Please choose your stock no. 3 from the following stock list:
PYPL
TSLA
U
AMD
NFLX
NIO
ILMN
GOOG
Enter your stock: MSFT
Enter your stock: ILMN
*********************************************

*********************************************
Please choose your stock no. 4 from the following stock list:
PYPL
TSLA
U
AMD
NFLX
NIO
GOOG
Enter your stock: PYPL
*********************************************

****************************

### 2. RSI compute method, RSI moving window range, and chart's start/end date

In [3]:
# instead of the traditional loops -- trying out new library!
# -------------------------------------------------------------

# RSI compute method: 
method = pyip.inputMenu(choices = ['SMA','EWMA'],
                        prompt = 'Enter your preferred method to compute the Relative Strength Index (RSI) value:\n' )
                        # method output is auto-capitalized

# RSI moving window range - the default is min of 14 days :)
period = pyip.inputInt(prompt = 'Enter the moving window range: (default: minimum of 14 days) ', min=14)

# customized function to check date validity:
first_date = date.today()-timedelta(days=365)    # based on the 1-year date range returned
last_date = date.today()-timedelta(days=1)       # yesterday's date

def check_start_date(date_in):
    date_in = datetime.strptime(date_in, "%Y-%m-%d").date()
    if date_in < first_date or date_in > last_date:
        raise Exception('WARNING: date input is not within range!')
        
def check_end_date(date_in):
    first_date = datetime.strptime(start_date,"%Y-%m-%d").date()    # the start date user entered
    date_in = datetime.strptime(date_in, "%Y-%m-%d").date()
    if date_in < first_date or date_in > last_date:
        raise Exception('WARNING: date input is not within range!')

# prompt message for date input        
prompt_s = f'Please enter a start date for the chart between {first_date} and {last_date} (format YYYY-MM-DD): '
start_date = pyip.inputCustom(check_start_date, prompt=prompt_s)

prompt_e = f'Please enter an end date for the chart between {start_date} and {last_date} (format YYYY-MM-DD): '
end_date = pyip.inputCustom(check_end_date, prompt=prompt_e)

Enter your preferred method to compute the Relative Strength Index (RSI) value:
* SMA
* EWMA
ssa
'ssa' is not a valid choice.
Enter your preferred method to compute the Relative Strength Index (RSI) value:
* SMA
* EWMA
EWMA
Enter the moving window range: (default: minimum of 14 days) 12
Number must be at minimum 14.
Enter the moving window range: (default: minimum of 14 days) 14
Please enter a start date for the chart between 2020-10-10 and 2021-10-09 (format YYYY-MM-DD): 2019-12-31
Please enter a start date for the chart between 2020-10-10 and 2021-10-09 (format YYYY-MM-DD): 2020-11-30
Please enter an end date for the chart between 2020-11-30 and 2021-10-09 (format YYYY-MM-DD): 2021-08-31


## Fetching Data from Yahoo! Finance via RapidAPI:

In [4]:
# function to fetch raw data

def fetch_rawdata(stock):
    # to fetch data from RapidApi using requests
    url = "https://yh-finance.p.rapidapi.com/stock/v3/get-historical-data"
    headers = {
        'x-rapidapi-host': "yh-finance.p.rapidapi.com",
        'x-rapidapi-key': "xxxx--your--key--here--xxxx"
    }
    querystring = {"symbol":{stock},"region":"US"}
    response = requests.request("GET", url, headers=headers, params=querystring)
    if (response.status_code == 200):
        response_dict = response.json()
    else:
        print('Data fetching failed..')
        
    df = pd.DataFrame(response_dict['prices'])
    df['date'] = df['date'].apply(lambda x: datetime.fromtimestamp(x).strftime("%Y-%m-%d"))
    
    return df

In [5]:
# function to calculate RSI

def RSI(df, period=14, method="SMA"):
    delta = df["close"].diff()
    up, down = delta.copy(), delta.copy()
    up[up < 0] = 0
    down[down > 0] = 0
       
    if method == "SMA":
        # using Smooth Moving Average
        gain = up.rolling(period).mean()
        loss = down.abs().rolling(period).mean()
    else:
        # using Exponential Weighted Moving Average
        gain = up.ewm(span=period).mean()
        loss = down.abs().ewm(span=period).mean()
    RS = gain / loss
    df['RSI'] =  pd.Series(100 - (100 / (1 + RS)), name="RSI")
    
    return df

In [6]:
# create a new dataframe to store all relevant data
col_names = ['date'] + user_option    # the stock list from user input
df_f = pd.DataFrame(columns=col_names)

for stock in user_option:
    df = fetch_rawdata(stock)
    df = RSI(df, period=14, method='SMA')
    df_f[stock] = df['RSI']
df_f['date'] = df['date']

## Extract the dataframe of interest date range:

In [7]:
# the full dataframe fetched!
df_f

,date,SQ,AMZN,ILMN,PYPL,U
0,2021-10-08,NaN,NaN,NaN,NaN,NaN
1,2021-10-07,NaN,NaN,NaN,NaN,NaN
2,2021-10-06,NaN,NaN,NaN,NaN,NaN
3,2021-10-05,NaN,NaN,NaN,NaN,NaN
4,2021-10-04,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...
247,2020-10-15,64.246921,55.375672,57.180771,55.423465,39.437412
248,2020-10-14,80.012281,71.330567,71.377511,67.603979,43.723683
249,2020-10-13,86.121356,78.329682,79.162960,65.668144,48.378294
250,2020-10-12,78.187212,77.333681,80.859219,61.249261,46.165935


In [8]:
# extracting the date range interested
df_frame = df_f[(df_f['date'] >= start_date) & (df_f['date'] <= end_date)]
df_frame

,date,SQ,AMZN,ILMN,PYPL,U
27,2021-08-31,68.596154,68.438638,71.530154,72.347859,44.121719
28,2021-08-30,72.655136,58.619173,69.411499,71.753379,48.158212
29,2021-08-27,63.219407,33.737620,68.490804,52.145771,25.428798
30,2021-08-26,56.848258,25.740830,70.577788,39.482568,24.339766
31,2021-08-25,68.565780,25.427808,77.865065,45.060456,37.719604
...,...,...,...,...,...,...
212,2020-12-04,34.598314,48.406140,36.495031,27.980584,41.911594
213,2020-12-03,30.822029,50.222067,34.326298,25.010023,37.435142
214,2020-12-02,15.996560,49.538966,27.235634,17.612547,32.437598
215,2020-12-01,20.168229,52.041256,28.018623,26.958204,39.716550


## Plotting of chart using Plotly:

In [9]:
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot

import cufflinks as cf
init_notebook_mode(connected=True)     # for jupyter notebook
cf.go_offline()

In [10]:
fig = df_frame.iplot(
    x='date',
    y=user_option,
    xTitle='date',
    yTitle='RSI Value',
    asFigure=True,
)
fig.add_hline(y=70,
              line_width=1,
              line_dash='dot',
              annotation_text="Overbought", 
              annotation_position="top right",
)

fig.add_hline(y=30,
              line_width=1,
              line_dash='dot',
              annotation_text="Oversold", 
              annotation_position="bottom right",
)
fig.update_layout(modebar_add=[
    "v1hovermode",
    "toggleSpikelines",
])

### another method of doing this: with plotly.express

In [11]:
import plotly.express as px

In [12]:
# melting the dataframe
df_frame_melt = df_frame.melt(id_vars='date').rename(columns={'variable':'stock','value':'RSI'})
display(df_frame_melt)

,date,stock,RSI
0,2021-08-31,SQ,68.596154
1,2021-08-30,SQ,72.655136
2,2021-08-27,SQ,63.219407
3,2021-08-26,SQ,56.848258
4,2021-08-25,SQ,68.565780
...,...,...,...
945,2020-12-04,U,41.911594
946,2020-12-03,U,37.435142
947,2020-12-02,U,32.437598
948,2020-12-01,U,39.716550


In [13]:
fig = px.line(df_frame_melt,
              x='date',
              y='RSI',
              template='plotly_white',
              color='stock',
)

fig.add_hline(y=70,
              line_width=1,
              line_dash='dot',
              annotation_text="Overbought", 
              annotation_position="top right",
)

fig.add_hline(y=30,
              line_width=1,
              line_dash='dot',
              annotation_text="Oversold", 
              annotation_position="bottom right",
)

## Appendix - trying out different APIs

### 1. Quandl
- able to specifiy the date range however data fetched only up to date **2018-03-27**
- source here: https://rapidapi.com/brianiswu/api/quandl1/

In [14]:
def fetch_rawdata_api(stock):
    # to fetch data from RapidApi using requests
    url = f"https://quandl1.p.rapidapi.com/datasets/WIKI/{stock}.json"
    querystring = {
        "column_index":"4",
        "end_date":end_date,
        "start_date":start_date,
        "collapse":"daily",
        "order":"desc",
        "transform":"rdiff"}
    headers = {
        'x-rapidapi-host': "quandl1.p.rapidapi.com",
        'x-rapidapi-key': "xxxx--your--key--here--xxxx"}
    
    response = requests.request("GET", url, headers=headers, params=querystring)
    if (response.status_code == 200):
        response_dict = response.json()
    else:
        print('Data fetching failed..')
    df = pd.DataFrame(response_dict['dataset']['data'], columns=['date','delta'])
    # data fetched includes date and the closing price difference
    return df

In [15]:
# example data:
stock='MSFT'
start_date = '2011-12-31'
end_date = datetime.strftime(date.today(), '%Y-%m-%d')

In [16]:
df_quandl = fetch_rawdata_api(stock)
display(df_quandl.head())
display(df_quandl.tail())

,date,delta
0,2018-03-27,-0.045959
1,2018-03-26,0.075705
2,2018-03-23,-0.029068
3,2018-03-22,-0.029087
4,2018-03-21,-0.006979


,date,delta
1561,2012-01-10,0.003605
1562,2012-01-09,-0.012987
1563,2012-01-06,0.015354
1564,2012-01-05,0.010219
1565,2012-01-04,0.023725


Note: 
1. slight adjustment in the proceding functions also need to be done as the data fetched only up to **2018-03-27**
2. the first date is different from selection too, data source may not be complete

### 2. Alphawave

- API for 5 years data, but can't directly specify date range upon calling
- source here: https://rapidapi.com/alphawave/api/stock-prices2/

In [17]:
# list of stocks:
input_list = ['PYPL','TSLA','NFLX','AMD','MSFT','SQ','AAPL','ILMN','AMZN','GOOG']
user_option = []


# get user input - to get the number of stocks, from 1 to 10
while True:
    num = input('Enter the number of stocks (from 1 to 10): ')
    try:
        num = int(num)
    except ValueError:
        print('WARNING: please enter value ranges from 1 to 10 only!')
        continue
    if (num<1) or (num>10):
        print('WARNING: please enter value ranges from 1 to 10 only!')
        continue
    break

    
# get user input - to get the list of stocks
if num == 10:
    user_option = input_list
else:
    for i in range(1, num+1):
        print('*'*45)
        print(f'Please choose your stock no. {i} from the following stock list:')
        for stock in input_list:
            print(stock)
        while True:
            option = input('Enter your stock: ')
            try:
                input_list.remove(option)
                user_option.append(option)
                print('*'*45)
                print()
            except ValueError:
                print('WARNING: please only enter stock listed from the list above!')
                continue
            break
            
print(f'This is your list of stocks: {user_option}')

Enter the number of stocks (from 1 to 10): 10
This is your list of stocks: ['PYPL', 'TSLA', 'NFLX', 'AMD', 'MSFT', 'SQ', 'AAPL', 'ILMN', 'AMZN', 'GOOG']


In [23]:
# instead of the traditional loops -- # trying out new library!
# -------------------------------------------------------------

# RSI compute method: 
method = pyip.inputMenu(choices = ['SMA','EWMA'],
                        prompt = 'Enter your preferred method to compute the Relative Strength Index (RSI) value:\n' )
                        # method output is auto-capitalized

# RSI moving window range - the default is min of 14 days :)
period = pyip.inputInt(prompt = 'Enter the moving window range: (default: minimum of 14 days) ', min=14)

# set the chart date range -- 5 years from 2016
first_date = date.today() - timedelta(days = 5*365-1)  # <-- 5-year-ago's date (approx. value)
last_date = date.today()                               # <-- today's date

# customized function to check date validity:
def check_start_date(date_in):
    date_in = datetime.strptime(date_in, "%Y-%m-%d").date()
    if date_in < first_date or date_in > last_date:
        raise Exception('WARNING: date input is not within range!')
        
def check_end_date(date_in):
    first_date = datetime.strptime(start_date,"%Y-%m-%d").date()
    date_in = datetime.strptime(date_in, "%Y-%m-%d").date()
    if date_in < first_date or date_in > last_date:
        raise Exception('WARNING: date input is not within range!')

# prompt message for date input        
prompt_s = f'Please enter a start date for the chart between {first_date} and {last_date} (format YYYY-MM-DD): '
start_date = pyip.inputCustom(check_start_date, prompt=prompt_s)

prompt_e = f'Please enter an end date for the chart between {start_date} and {last_date} (format YYYY-MM-DD): '
end_date = pyip.inputCustom(check_end_date, prompt=prompt_e)

Enter your preferred method to compute the Relative Strength Index (RSI) value:
* SMA
* EWMA
sma
Enter the moving window range: (default: minimum of 14 days) 14
Please enter a start date for the chart between 2016-10-12 and 2021-10-10 (format YYYY-MM-DD): 2016-10-12
Please enter an end date for the chart between 2016-10-12 and 2021-10-10 (format YYYY-MM-DD): 2021-10-10


In [19]:
# slight modification in the function:

def fetch_rawdata_api(stock):
    # to fetch data from RapidApi using requests
    url = "https://stock-prices2.p.rapidapi.com/api/v1/resources/stock-prices/5y"
    querystring = {"ticker":stock}
    headers = {
        'x-rapidapi-host': "stock-prices2.p.rapidapi.com",
        'x-rapidapi-key': "xxxx--your--key--here--xxxx"
    }
    response = requests.request("GET", url, headers=headers, params=querystring)
    if (response.status_code == 200):
        response_dict = response.json()
    else:
        print('Data fetching failed..')
    df = pd.DataFrame(response_dict).T.reset_index().rename(columns={'index':'Date'}) 
    df = df.drop(columns=['Dividends', 'Stock Splits', 'Volume'])    # extra columns not needed in this case
    return df

In [20]:
# function to calculate RSI

def RSI(df, period=14, method="SMA"):
    delta = df["Close"].diff()
    up, down = delta.copy(), delta.copy()
    up[up < 0] = 0
    down[down > 0] = 0
       
    if method == "SMA":
        # using Smooth Moving Average
        gain = up.rolling(period).mean()
        loss = down.abs().rolling(period).mean()
    else:
        # using Exponential Weighted Moving Average
        gain = up.ewm(span=period).mean()
        loss = down.abs().ewm(span=period).mean()
    RS = gain / loss
    df['RSI'] =  pd.Series(100 - (100 / (1 + RS)), name="RSI")
    
    return df

In [21]:
# create a new dataframe to store all relevant data
col_names = ['Date'] + user_option    # the stock list from user input
df_f = pd.DataFrame(columns=col_names)

for stock in user_option:
    df = fetch_rawdata_api(stock)
    df = RSI(df, period=14, method='SMA')
    df_f[stock] = df['RSI']
df_f['Date'] = df['Date']

In [22]:
# the dataframe created!
df_f

,Date,PYPL,TSLA,NFLX,AMD,MSFT,SQ,AAPL,ILMN,AMZN,GOOG
0,2016-10-10,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2016-10-11,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2016-10-12,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2016-10-13,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2016-10-14,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...
1254,2021-10-04,27.005591,63.671939,63.200223,39.493171,34.376159,36.812070,30.393004,3.354500,24.002972,32.057895
1255,2021-10-05,32.637167,59.899248,71.065852,42.978883,35.129649,42.618604,33.472441,11.790113,24.818648,33.571071
1256,2021-10-06,35.998819,60.217356,71.219924,45.425531,39.551331,40.458836,36.204792,16.850910,28.836054,37.385661
1257,2021-10-07,39.579954,62.691558,66.557570,54.482036,45.380950,46.436639,44.022425,27.178043,35.424364,45.741250


In [24]:
df_frame = df_f[(df_f['Date'] >= start_date) & (df_f['Date'] <= end_date)]

In [25]:
fig = df_frame.iplot(
    x='Date',
    y=user_option,
    xTitle='Date',
    yTitle='RSI Value',
    asFigure=True,
)
fig.add_hline(y=70,
              line_width=1,
              line_dash='dot',
              annotation_text="Overbought", 
              annotation_position="top right",
)

fig.add_hline(y=30,
              line_width=1,
              line_dash='dot',
              annotation_text="Oversold", 
              annotation_position="bottom right",
)
# adding in range slider for better viewing of chart
fig.update_xaxes(range=[start_date, end_date],
                 rangeslider_visible=True,
                 rangeselector=dict(
                     buttons=list([
                         dict(count=1, label="1m", step="month", stepmode="backward"),
                         dict(count=6, label="6m", step="month", stepmode="backward"),
                         dict(count=1, label="YTD", step="year", stepmode="todate"),
                         dict(count=1, label="1y", step="year", stepmode="backward"),
                         dict(step="all")
                     ])
                 )
)
fig.update_layout(modebar_add=[
    "v1hovermode",
    "toggleSpikelines",
])